<a href="https://colab.research.google.com/github/acse-rt1120/test/blob/main/no_means_network_(week7).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

from livelossplot import PlotLosses
from pycm import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
import torch  # Pytorch
import torch.nn as nn  # Neural network module
import torch.nn.functional as fn  # Function module
from torchvision import datasets  # Datasets from torchvision
from torchvision import transforms  # Transforms from torchvision

import matplotlib.pyplot as plt  # Plotting using matplotlib
import numpy as np  # Numpy
from numpy import linalg
device = 'cuda'  # Set out device to GPU

print('done')  # Let me know this cell has finished

In [ ]:
# MNIST Test dataset and dataloader declaration
batch_size = 64
data = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([transforms.ToTensor(),])
                   ),
                   batch_size=batch_size, shuffle=True) # Load MNIST. Use the Data Loader to shuffle and batch images

images, labels = next(iter(data)) # A trick for getting a batch out of the dataloader object
plt.imshow(images[0].squeeze()) # Show the first image from the batch
plt.show()
batch_num = len(data)
data_size = len(data)*batch_size
print('number of batches = '+str(len(data))) # Print num batches
print('number of images = '+str(len(data)*batch_size)) # Print num images

In [ ]:
import torch
import numpy as np
x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])
x_data_torch = torch.from_numpy(x_data).float() 

y_data = np.array([0,1,2,0,0,2])
y_data_torch = torch.from_numpy(y_data)

num_features = 28*28
num_classes = 10
n_hidden_1 = 25


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
W1 = torch.randn(num_features, n_hidden_1, requires_grad=True)
B1 = torch.randn(n_hidden_1, requires_grad=True)

Wout = torch.randn(n_hidden_1, num_classes, requires_grad=True)
Bout = torch.randn(num_classes, requires_grad=True)

learning_rate=0.01
no_of_epochs = 10

for epoch in range(no_of_epochs):     #explicitly defne the forward and backward
    z1 = torch.add(torch.matmul(x_data_torch, W1), B1) 
    Zout = torch.add(torch.matmul(nn.ReLU(z1), Wout), Bout)

    # criterion = nn.CrossEntropyLoss()
    # loss = criterion(a2, y) 
    log_softmax = F.log_softmax(Zout,dim=1)
    loss = F.nll_loss(log_softmax, y_data_torch)

    loss.backward()
    with torch.no_grad():
        W1.data -= learning_rate*W1.grad.data
        B1.data -= learning_rate*B1.grad.data
        Wout.data -= learning_rate*Wout.grad.data
        Bout.data -= learning_rate*Bout.grad.data

    W1.grad.data.zero_()
    B1.grad.data.zero_()
    Wout.grad.data.zero_()
    Bout.grad.data.zero_()
    


    if epoch % 200 == 199: 
        with torch.no_grad():
            z1 = torch.add(torch.matmul(x_data_torch ,W1),B1)
            Zout = torch.add(torch.matmul(nn.ReLU(z1) ,Wout),Bout)
            predicted = torch.argmax(Zout, 1)
            train_acc = accuracy_score(predicted.numpy(),y_data)
            print('Epoch: %d, loss: %.4f, train_acc: %.3f' %(epoch + 1, loss.item() , train_acc))
print("Finished")
# Result
print('Predicted :', predicted.numpy())
print('Truth :', y_data)
print('Accuracy : %.2f' %train_acc)


Epoch: 200, loss: 0.5847, train_acc: 0.833
Epoch: 400, loss: 0.4327, train_acc: 0.833
Epoch: 600, loss: 0.3225, train_acc: 0.833
Epoch: 800, loss: 0.2372, train_acc: 1.000
Epoch: 1000, loss: 0.1740, train_acc: 1.000
Finished
Predicted : [0 1 2 0 0 2]
Truth : [0 1 2 0 0 2]
Accuracy : 1.00


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score


class ModelWithHiddenLayer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ModelWithHiddenLayer, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        z1 = self.linear1(x)
        Zout = self.linear2(F.relu(z1))
        return Zout

model = ModelWithHiddenLayer(num_features, n_hidden_1, num_classes)

learning_rate=0.01
no_of_epochs = 1000

# If you apply Pytorch’s CrossEntropyLoss to your output layer,
# you get the same result as applying Pytorch’s NLLLoss to a LogSoftmax layer added after your original output layer.
criterion = nn.CrossEntropyLoss() 
optimiser = optim.SGD(model.parameters(), lr=learning_rate)


In [ ]:
print("Model's state_dict:")
layers_num = -1
layer_size = []
for param_tensor in model.state_dict():
    layers_num += 1
    layer_size.append(list(model.state_dict()[param_tensor].size()))
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
linear1.weight 	 torch.Size([5, 2])
linear1.bias 	 torch.Size([5])
linear2.weight 	 torch.Size([3, 5])
linear2.bias 	 torch.Size([3])


In [ ]:
for epoch in range(no_of_epochs):  # loop over the dataset multiple times

    # get the inputs
    inputs = x_data_torch
    labels = y_data_torch

    model.train()
    # zero the parameter gradients
    for param in model.parameters():
      param.grad.zero_()
    # linear1.weight.grad.data.zero_()
    # linear1.bias.grad.data.zero_()
    # linear2.weight.grad.data.zero_()
    # linear2.bias.grad.data.zero_()

    # forward + backward + optimize
    outputs = model(inputs)
    loss = criterion(outputs, labels) # We don't need to calcualte logsoftmax here
    loss.backward()
    with torch.no_grad():
        linear1.weight.data -= learning_rate*linear1.weight.grad.data
        linear1.bias.data -= learning_rate*linear1.bias.grad.data
        linear2.weight.data -= learning_rate*linear2.weight.grad.data
        linear2.bias.data -= learning_rate*linear2.bias.grad.data

    # print statistics
    if epoch % 200 == 199:    # print every 200 epochs
        model.eval()
        pred_outputs = model(inputs)
        predicted = torch.argmax(pred_outputs, 1)
        train_acc = accuracy_score(predicted.numpy(),y_data)
        print('%d, loss: %.4f, train_acc: %.4f' %(epoch + 1, loss.item(), train_acc))

print('Finished Training')

# Result
pred_outputs = model(inputs)
_, predicted = torch.max(pred_outputs, 1)
print('Predicted :', predicted.numpy())
print('Truth :', y_data)

train_acc = accuracy_score(predicted.numpy(),y_data)
print('Accuracy : %.2f' %train_acc)


AttributeError: ignored

In [ ]:
for epoch in range(no_of_epochs):     #explicitly defne the forward and backward
    z1 = torch.add(torch.matmul(x_data_torch, W1), B1) 
    Zout = torch.add(torch.matmul(F.relu(z1), Wout), Bout) 

    log_softmax = F.log_softmax(Zout,dim=1)
    loss = F.nll_loss(log_softmax, y_data_torch)

    loss.backward()
    with torch.no_grad():
        W1.data -= learning_rate*W1.grad.data
        B1.data -= learning_rate*B1.grad.data
        Wout.data -= learning_rate*Wout.grad.data
        Bout.data -= learning_rate*Bout.grad.data

    W1.grad.data.zero_()
    B1.grad.data.zero_()
    Wout.grad.data.zero_()
    Bout.grad.data.zero_()
    


    if epoch % 200 == 199: 
        with torch.no_grad():
            z1 = torch.add(torch.matmul(x_data_torch ,W1),B1)
            Zout = torch.add(torch.matmul(F.relu(z1) ,Wout),Bout)
            predicted = torch.argmax(Zout, 1)
            train_acc = accuracy_score(predicted.numpy(),y_data)
            print('Epoch: %d, loss: %.4f, train_acc: %.3f' %(epoch + 1, loss.item() , train_acc))
print("Finished")
# Result
print('Predicted :', predicted.numpy())
print('Truth :', y_data)
print('Accuracy : %.2f' %train_acc)
